# Configuring IPython

## Finding configuration options

IPython has many configurable attributes. These can be viewed using the `-h` flag to the command line applications:

In [ ]:
!ipython -h

This is an important trick for finding out configuration info:
    
    $> ipython [subcommand] --help-all | grep [-C context] PATTERN

`--help-all` exposes everything configurable in IPython,
there is a good chance you will find what you are looking for.

A common configuration question is:

> how do I disable the "Do you really want to exit" message when quitting with `Ctrl-d`?

Well, logically this has to do with `exit`, so let's look for it:

In [ ]:
!ipython --help-all | GREP_COLOR='1;31;46' grep --color exit

Which shows me that I can disable the confirmation for a single IPython session with

    $> ipython --no-confirm-exit
    
or I can set the `TerminalInteractiveShell.confirm_exit=False` in a config file,
to have it be the default behavior.

## Configuration principles

Here are the design principles of the IPython configuration system: 

* Configuration is always done using class attributes
* Classes that have configurable attributes are subclasses of `Configurable`
* Attributes that are configurable are typed traitlets objects (`Bool`, `Unicode`, etc.) that have `config=True`
* In config files, configurable attributes can be set using the format `Class.attr_name=the_value`
* At the command line, configurable attributes can be set using the syntax `--Class.attr_name=the_value`
* At the command line, some attributes have shorthands of the form `--attr-name=value`
* Values set at the command line have higher priority than those set in config files

## The IPython Profile

IPython has a notion of 'profiles' - these are directories that live in your IPYTHONDIR,
which contain configuration and runtime information.

Let's create the default profile

In [ ]:
!ipython profile create newprofile

This creates a profile in your IPYTHONDIR (`ipython locate` is a quick way to see where your IPYTHONDIR is),
and populates it with automatically generated default config files.

In [ ]:
!ipython locate profile default
!ipython locate profile newprofile

You can skim

In [ ]:
profile = get_ipython().profile_dir.location
profile

In [ ]:
ls $profile

Let's peek at our config file

In [ ]:
pycat $profile/ipython_config.py

## Startup Files

Startup files are simple Python or IPython scripts
that are run whenever you start IPython.
These are a useful way to do super common imports,
or for building database connections to load on startup of a non-default profile.

We can use a startup file to ensure that our `%tic/toc` magics are always defined,
every time we start IPython.

In [ ]:
!ls $profile/startup

In [ ]:
!cat $profile/startup/README

Adding common imports, so we never have to forget them again

In [ ]:
#%%writefile $profile/startup/simpleimports.py
#
#import sys, os, time, re

**Restart the kernel** and then run the following cells immediately to verify these scripts have been executed:

In [ ]:
#sys

## Defining your own magic

As we have seen already, IPython has cell and line magics. You can define your own magics using any Python function and the `register_magic_function` method:

In [ ]:
from IPython.core.magic import (register_line_magic, register_cell_magic,
                                register_line_cell_magic)

In [ ]:
@register_line_magic
def sleep(line):
    """A simple function for sleeping"""
    import time
    t = float(line)
    time.sleep(t)

In [ ]:
%sleep 2

In [ ]:
%sleep?

### Cell Magic

**Cell magics** take two args:

1. the **line** on the same line of the magic 
2. the **cell** the multiline body of the cell after the first line

In [ ]:
@register_cell_magic
def dummy(line, cell):
    """dummy cell magic for displaying the line and cell it is passed"""
    print("line: %r" % line)
    print("cell: %r" % cell)

In [ ]:
%%dummy this is the line
this
is the
cell

For more details on declaring your own magics in more complex scenarios, [see our docs](http://ipython.org/ipython-doc/stable/interactive/reference.html#defining-your-own-magics).